# CODE REVIEW: CEP CODE COORDINATES

Review of [this process](https://github.com/justosbr/CEP_coordinates) intended to assign a unique location (latitude, longitude) to any given CEP code


In [1]:
import csv, re, psycopg2, os
from tqdm import tqdm
import pandas as pd
import numpy as np
from io import TextIOBase



---

## 1. File download

The process first downloads all available zip files containing unique addresses by area. Straightforward code hosted [here](https://github.com/justosbr/CEP_coordinates/blob/master/src/download_CNEFE.sh).

**The output of the process is a local `all.txt` file** which is the concatenation of all records across files

---

## 2. File to CSV

Python process to read `all.txt` file and creates two files fromt:

- `all.csv`: File containing the same records but in a more readable, csv format

In [2]:
pd.read_csv('data/cep_coordinates/all.csv', nrows=2)

,cod_uf,cod_municipio,cod_distrito,cod_subdistrito,cod_setor,sit_setor,tipo_logradouro,titulo_logradouro,nome_logradouro,num_logradouro,...,longitude,localidade,nulo,especie_endereco,id_establecimiento,indic_endereco,id_col_domicilio,num_quadra,num_face,CEP
0,11,15,5,0,1,1,RUA,NaN,ACRE,4305,...,NaN,BAIRRO REDONDO,NaN,1,NaN,NaN,NaN,5,3,76954000
1,11,15,5,0,1,1,RUA,NaN,ACRE,4328,...,NaN,BAIRRO REDONDO,NaN,1,NaN,NaN,NaN,4,1,76954000


- `census_tracks_to_ceps.csv`: File containing the aggregate of addresses within each geographical delimitation. This geographical unit is defined by the tuple (`cod_uf`, `cod_municipio`, `cod_distrito`, `cod_subdistrito`, `cod_setor`, `CEP`)

In [3]:
census_tracts_to_ceps = pd.read_csv('data/cep_coordinates/census_tracks_to_ceps.csv').drop(columns='Unnamed: 0')
census_tracts_to_ceps.head(2)
                                    

,cod_uf,cod_municipio,cod_distrito,cod_subdistrito,cod_setor,CEP,address_count
0,11,15,5,0,1,76954000.0,392
1,11,15,5,0,2,76954000.0,334


Process hosted [here](https://github.com/justosbr/CEP_coordinates/blob/master/src/process.py)


---

## 3. Obtaining CEP coordinates

This step is performed with a R process hosted [here](https://github.com/justosbr/CEP_coordinates/blob/master/src/process.R). **The output of the process is the Postgres table `susep.geo_info`.**

In this section we will aim at replicating the process with an independent Python implementation

### 3.1. Obtain census tract codes

First load and transform the data with that relates CEP codes and census tracts

In [4]:
# Load the data
data_census = pd.read_csv("data/cep_coordinates/census_tracks_to_ceps.csv").drop(columns='Unnamed: 0')
display(data_census.shape)
data_census = data_census[(data_census['CEP']!=0) & (data_census['CEP'].notna())]
display(data_census.shape)
display(data_census.head())
print('Any missing values?')
display(data_census.isna().sum())


(1301810, 7)

(1288478, 7)

,cod_uf,cod_municipio,cod_distrito,cod_subdistrito,cod_setor,CEP,address_count
0,11,15,5,0,1,76954000.0,392
1,11,15,5,0,2,76954000.0,334
2,11,15,5,0,3,76954000.0,255
3,11,15,5,0,4,76954000.0,111
4,11,15,5,0,5,76954000.0,424


Any missing values?


cod_uf             0
cod_municipio      0
cod_distrito       0
cod_subdistrito    0
cod_setor          0
CEP                0
address_count      0
dtype: int64

Composition of census tracts is clearly depicted [here](https://www.ibge.gov.br/geociencias/organizacao-do-territorio/malhas-territoriais/26565-malhas-de-setores-censitarios-divisoes-intramunicipais.html?=&t=saiba-mais-edicao) as a concatenation of diferent elements:

![alt text](images/setores_censitarios.png "Title")

Create a column with the census tract code from the info above

In [5]:
field_lengths = {
    'cod_uf': 2,
    'cod_municipio': 5,
    'cod_distrito': 2,
    'cod_subdistrito': 2,
    'cod_setor': 4}

# Transform fields
for field, field_length in field_lengths.items():
    data_census[field] = data_census[field].astype(str).str.zfill(field_length)
    
# Build the census tract code
data_census['census_tract_code'] = data_census.apply(lambda row: row['cod_uf']+row['cod_municipio']+row['cod_distrito']+row['cod_subdistrito']+row['cod_setor'], axis=1)
data_census.head(5)


,cod_uf,cod_municipio,cod_distrito,cod_subdistrito,cod_setor,CEP,address_count,census_tract_code
0,11,00015,05,00,0001,76954000.0,392,110001505000001
1,11,00015,05,00,0002,76954000.0,334,110001505000002
2,11,00015,05,00,0003,76954000.0,255,110001505000003
3,11,00015,05,00,0004,76954000.0,111,110001505000004
4,11,00015,05,00,0005,76954000.0,424,110001505000005


How do census tracts and CEPs relate?

In [6]:
print('Census tracts with the highest number of CEP codes:')
display(data_census.groupby('census_tract_code').CEP.nunique().sort_values(ascending=False).head(10))
print('CEP codes with the highest number of census tracts:')
display(data_census.groupby('CEP').census_tract_code.nunique().sort_values(ascending=False).head(10))


Census tracts with the highest number of CEP codes:


census_tract_code
320500210000003    169
521250105000148    144
170210905000104     97
521250105000147     96
170210905000115     90
350760505000138     76
521250105000155     73
521760905000079     68
172100005000067     61
317020605000502     60
Name: CEP, dtype: int64

CEP codes with the highest number of census tracts:


CEP
9999999.0     2831
99999999.0    2467
60000000.0    1180
65000000.0     879
64000000.0     571
74000000.0     548
88888888.0     542
40000000.0     536
78000000.0     461
24800000.0     447
Name: census_tract_code, dtype: int64

### 3.2. Obtain census tract centroids

We can obtain census tract centroids from the data we already have in Postgres

In [7]:
# Establish connection and create its cursor
try: 
    conn = psycopg2.connect(f"host={os.environ['AURORA_POSTGRES_HOST']} dbname={os.environ['AURORA_POSTGRES_DATABASE']} user={os.environ['AURORA_POSTGRES_USERNAME']} password={os.environ['AURORA_POSTGRES_PWD']}")
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
    

Get centroids of census tracts

In [8]:
statement = """
    SELECT 
        cd_geocodi,
        ST_X(ST_Centroid(census_tract_geom)) as centroid_lon,
        ST_Y(ST_Centroid(census_tract_geom)) as centroid_lat
    FROM susep.census_tract_detail
    """
cur.execute(statement)
centroids = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description]).rename(columns={'cd_geocodi': 'census_tract_code'})
centroids.head()


,census_tract_code,centroid_lon,centroid_lat
0,350635905000052,-46.119640,-23.797849
1,350690405000020,-48.249988,-23.185557
2,350745605000008,-48.992364,-22.680905
3,350750610000012,-48.754408,-22.907515
4,350750615000008,-48.305934,-22.719156


### 3.3. Merge the datasets

In [9]:
print('Census tracts for which we have a geometry:', len(set(centroids.census_tract_code)))
print('Census tracts on which we have any address:', len(set(data_census.census_tract_code)))
print('Total of census tracts with a geometry but no addresses:', len(set(centroids.census_tract_code)-set(data_census.census_tract_code)))


Census tracts for which we have a geometry: 68296
Census tracts on which we have any address: 309913
Total of census tracts with a geometry but no addresses: 1736


In [10]:
merge_data = centroids.merge(data_census, on='census_tract_code', how='left')
merge_data.head()

,census_tract_code,centroid_lon,centroid_lat,cod_uf,cod_municipio,cod_distrito,cod_subdistrito,cod_setor,CEP,address_count
0,350635905000052,-46.119640,-23.797849,35,06359,05,00,0052,11250000.0,57.0
1,350690405000020,-48.249988,-23.185557,35,06904,05,00,0020,18590000.0,301.0
2,350745605000008,-48.992364,-22.680905,35,07456,05,00,0008,18675.0,214.0
3,350750610000012,-48.754408,-22.907515,35,07506,10,00,0012,18603970.0,227.0
4,350750615000008,-48.305934,-22.719156,35,07506,15,00,0008,1860000.0,60.0


To summarize so far:
- We have census tracts with a centroid to each one
- We have a certain number of addresses for each (`census_tract_code`, `CEP`) tuple
- To arrive to `CEP` coordinates, what we want is to **find a way to use the `census_tract_code` coordinates available to each `CEP` to obtain one unique location. It is proposed to use a weighted average**

### 3.4. Obtaining one coordinate pair for each CEP

Weighted average of census tract coordinates using the number of available addresses

In [11]:
final_data = merge_data.groupby('CEP').apply(lambda group: (np.average(group.centroid_lon, weights=group.address_count), 
                                                            np.average(group.centroid_lat, weights=group.address_count))).reset_index()

final_data['cep_lon'] = final_data[0].apply(lambda x: x[0])
final_data['cep_lat'] = final_data[0].apply(lambda x: x[1])
final_data = final_data.drop(columns=0)
final_data.head()



,CEP,cep_lon,cep_lat
0,1.0,-46.595471,-23.524142
1,2.0,-46.385094,-23.984590
2,3.0,-46.740471,-23.836161
3,4.0,-45.845959,-23.180913
4,7.0,-46.512144,-23.400021


Now a single `census_tract_code` is mapped to each CEP, being the chosen one the argmax of address count for that given CEP

In [12]:
census_tract_code_choice = merge_data.sort_values('address_count').groupby('CEP').last().reset_index()[['CEP', 'census_tract_code']]
census_tract_code_choice.head()


,CEP,census_tract_code
0,1.0,355030876000068
1,2.0,355100905000128
2,3.0,355030855000256
3,4.0,354990405000191
4,7.0,351880005000958


And finally get the address count as the summation of all the addresses across census tracts for that CEP

In [13]:
address_counts = merge_data.groupby('CEP').address_count.sum().reset_index()
address_counts.head()

,CEP,address_count
0,1.0,910.0
1,2.0,4.0
2,3.0,1.0
3,4.0,9.0
4,7.0,39.0


Add this info

In [14]:
final_data = final_data.merge(census_tract_code_choice, 
                              on='CEP', 
                              how='left').merge(address_counts,
                                                on='CEP',
                                                how='left')
final_data.head()

,CEP,cep_lon,cep_lat,census_tract_code,address_count
0,1.0,-46.595471,-23.524142,355030876000068,910.0
1,2.0,-46.385094,-23.984590,355100905000128,4.0
2,3.0,-46.740471,-23.836161,355030855000256,1.0
3,4.0,-45.845959,-23.180913,354990405000191,9.0
4,7.0,-46.512144,-23.400021,351880005000958,39.0


---

## 4. Comparing against `susep.geo_info`

Two tables to compare

In [15]:
final_data.head()

,CEP,cep_lon,cep_lat,census_tract_code,address_count
0,1.0,-46.595471,-23.524142,355030876000068,910.0
1,2.0,-46.385094,-23.984590,355100905000128,4.0
2,3.0,-46.740471,-23.836161,355030855000256,1.0
3,4.0,-45.845959,-23.180913,354990405000191,9.0
4,7.0,-46.512144,-23.400021,351880005000958,39.0


In [16]:
cur.execute("SELECT * FROM susep.geo_info")
geo_info = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
geo_info['CEP'] = geo_info['CEP'].astype(int)
geo_info.head()


,Id,CEP,code_tract,address_count,lat,lon,reg_susep,cep_inicial,cidade,cep_final,cod_reg,reg_decirc,cep_ini,cep_fim
0,1,76954000,110001520000004,12060,-12.047040,-62.051736,39.0,76740,FAINA,76999,48.0,DEMAIS REGI�ES DE GOI�S,76740,76999
1,777,76873868,110002305000060,11,-9.900819,-63.026831,39.0,76740,FAINA,76999,48.0,DEMAIS REGI�ES DE GOI�S,76740,76999
2,2,76955000,110007205000011,76,-12.983041,-60.952011,39.0,76740,FAINA,76999,48.0,DEMAIS REGI�ES DE GOI�S,76740,76999
3,1553,76965748,110004905000051,8,-11.436342,-61.436585,39.0,76740,FAINA,76999,48.0,DEMAIS REGI�ES DE GOI�S,76740,76999
4,3,76594000,110001505000018,9,-11.928932,-61.990097,39.0,76590,SAO MIGUEL DO ARAGUAIA,76599,48.0,DEMAIS REGI�ES DE GOI�S,76590,76599


In terms of coverage of CEPS, `susep.geo_info` contains all CEPs we have just generate info for. Filter these CEPs

In [17]:
display(len(set(geo_info.CEP)))
display(len(set(final_data.CEP)))
display(len(set(final_data.CEP)-set(geo_info.CEP)))
geo_info = geo_info[geo_info['CEP'].isin(set(final_data.CEP))].copy()


562192

182307

0

Compare coordinate pairs for some examples

In [18]:
final_data[final_data['CEP']==7]

,CEP,cep_lon,cep_lat,census_tract_code,address_count
4,7.0,-46.512144,-23.400021,351880005000958,39.0


In [19]:
geo_info[geo_info['CEP']==7]

,Id,CEP,code_tract,address_count,lat,lon,reg_susep,cep_inicial,cidade,cep_final,cod_reg,reg_decirc,cep_ini,cep_fim
160851,177320,7,351880005000958,45,-22.937053,-46.9919,NaN,None,None,None,NaN,None,None,None


In [20]:
merge_data[merge_data['CEP']==7]

,census_tract_code,centroid_lon,centroid_lat,cod_uf,cod_municipio,cod_distrito,cod_subdistrito,cod_setor,CEP,address_count
91616,351880005000958,-46.512144,-23.400021,35,18800,05,00,0958,7.0,39.0


In [21]:
data_census[data_census['CEP']==7]

,cod_uf,cod_municipio,cod_distrito,cod_subdistrito,cod_setor,CEP,address_count,census_tract_code
457182,32,05002,10,00,0003,7.0,1,320500210000003
680511,35,18800,05,00,0958,7.0,39,351880005000958
1214511,50,04403,20,00,0002,7.0,5,500440320000002


In `susep.geo_info` we have aggregated locations from census tracts that are not in Sao Paulo, which is likely wrong